# Solution
---

In [15]:
from utils import sigmoid, relu, slice_dataset
import numpy as np  
import matplotlib.pyplot as plt

In [16]:
# mu = float(input("Enter mean:"))
# print('Mean:', mu)

# sigma = float(input("Enter standard deviation:"))
# print('Standard Deviation:', sigma)

mu = 0.0
sigma = 0.1

# make normal distribution for user input parameters
dataset = np.random.normal(mu, sigma, 10)

# count, bins, ignored = plt.hist(dataset, 30, density=True)
# plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
#                np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
#          linewidth=2, color='r')
# plt.show()

print(slice_dataset(dataset, -2*sigma, 2*sigma))

0
